In [1]:
import pandas as pd
import ast
import numpy as np

# --- 1. Load and Clean the Dataset ---
file_path = "C:/Users/SHUBHAM/Desktop/sample_project/internship/TRADES_CopyTr_90D_ROI.csv"
df = pd.read_csv(file_path)

In [2]:
# Drop rows where Trade_History is missing
df = df.dropna(subset=['Trade_History'])

In [3]:
# Convert Trade_History from a JSON-like string to a list of dictionaries
df['Trade_History'] = df['Trade_History'].apply(ast.literal_eval)

In [4]:
# --- 2. Calculate Metrics for Each Account ---
metrics_list = []

for index, row in df.iterrows():
    port_id = row['Port_IDs']
    trades = row['Trade_History']
    
    # If trades have a 'time' key, sort by time
    if trades and isinstance(trades[0], dict) and 'time' in trades[0]:
        trades = sorted(trades, key=lambda x: x.get('time', 0))
    
    pnl = 0.0
    total_positions = 0
    win_positions = 0
    trade_returns = []
    cumulative_profit = []
    running_profit = 0.0
    
    # Process each trade in the account
    for trade in trades:
        profit = float(trade.get('realizedProfit', 0))
        qty = float(trade.get('quantity', 0))  # 'quantity' represents money used
        
        # Calculate per-trade return if quantity is nonzero
        if qty != 0:
            ret = profit / qty
            trade_returns.append(ret)
        
        pnl += profit
        total_positions += 1
        if profit > 0:
            win_positions += 1
        
        running_profit += profit
        cumulative_profit.append(running_profit)
    
    # ROI calculation: using total traded money (sum of quantity)
    total_quantity = sum(float(trade.get('quantity', 0)) for trade in trades)
    roi = (pnl / total_quantity) if total_quantity != 0 else 0
    
    win_rate = win_positions / total_positions if total_positions != 0 else 0
    
    # Sharpe Ratio: mean return divided by standard deviation of returns
    if trade_returns:
        mean_return = np.mean(trade_returns)
        std_return = np.std(trade_returns)
        sharpe = (mean_return / std_return) if std_return != 0 else 0
    else:
        sharpe = 0
    
    # Maximum Drawdown (MDD) calculation:
    max_drawdown = 0
    peak = -float('inf')
    for cp in cumulative_profit:
        if cp > peak:
            peak = cp
        drawdown = peak - cp
        if drawdown > max_drawdown:
            max_drawdown = drawdown

    metrics_list.append({
        'Port_ID': port_id,
        'PnL': pnl,
        'ROI': roi,
        'Sharpe': sharpe,
        'MDD': max_drawdown,
        'Win_Rate': win_rate,
        'Win_Positions': win_positions,
        'Total_Positions': total_positions
    })

# Convert the list of dictionaries into a DataFrame
metrics_df = pd.DataFrame(metrics_list)

# --- 3. Ranking Algorithm ---
# Create a composite score:
# Higher ROI, PnL, Sharpe, and Win_Rate increase the score,
# while a higher MDD (indicating deeper drawdowns) reduces the score.
# Note: PnL is scaled by 0.001 as it may be a large number.
metrics_df['Score'] = (
    metrics_df['ROI'] +
    0.001 * metrics_df['PnL'] +
    metrics_df['Sharpe'] +
    metrics_df['Win_Rate'] -
    metrics_df['MDD']  # Penalize by maximum drawdown
)

# Sort accounts by the composite score (higher is better)
metrics_df = metrics_df.sort_values(by='Score', ascending=False)

# --- 4. Extract the Top 20 Accounts ---
top_20 = metrics_df.head(20)

# Display the calculated metrics and the top 20 list
print("Calculated Metrics (Sample):")
print(metrics_df.head())
print("\nTop 20 Accounts Based on Composite Score:")
print(top_20)


Calculated Metrics (Sample):
                Port_ID           PnL       ROI    Sharpe       MDD  Win_Rate  \
35  3986814617275053313  16337.461881  0.008716  1.288733  2.381947  0.782217   
60  3936410995029308417  10681.514000  0.017911  0.595816  0.000000  0.300000   
9   4029749871687083265   3662.136548  0.031705  0.719655  0.000000  0.428738   
21  3891020560590657281   2856.300564  0.017434  1.129692  0.000000  0.647597   
38  4036548677626662656   3296.876680  0.005617  0.736671  0.000000  0.462585   

    Win_Positions  Total_Positions      Score  
35           2780             3554  16.035181  
60              6               20  11.595240  
9             367              856   4.842235  
21            283              437   4.651024  
38             68              147   4.501749  

Top 20 Accounts Based on Composite Score:
                 Port_ID           PnL       ROI    Sharpe       MDD  \
35   3986814617275053313  16337.461881  0.008716  1.288733  2.381947   
60   3936

In [7]:
# --- 3. Ranking Algorithm ---
# Create a composite score:
# Higher ROI, PnL, Sharpe, and Win_Rate increase the score,
# while a higher MDD (indicating deeper drawdowns) reduces the score.
# Note: PnL is scaled by 0.001 as it may be a large number.
metrics_df['Score'] = (
    metrics_df['ROI'] +
    0.001 * metrics_df['PnL'] +
    metrics_df['Sharpe'] +
    metrics_df['Win_Rate'] -
    metrics_df['MDD']  # Penalize by maximum drawdown
)

# Sort accounts by the composite score (higher is better)
metrics_df = metrics_df.sort_values(by='Score', ascending=False)



In [8]:
# --- 4. Extract the Top 20 Accounts ---
top_20 = metrics_df.head(20)

# Display the calculated metrics and the top 20 list
print("Calculated Metrics (Sample):")
print(metrics_df.head())
print("\nTop 20 Accounts Based on Composite Score:")
print(top_20)

Calculated Metrics (Sample):
                Port_ID           PnL       ROI    Sharpe       MDD  Win_Rate  \
35  3986814617275053313  16337.461881  0.008716  1.288733  2.381947  0.782217   
60  3936410995029308417  10681.514000  0.017911  0.595816  0.000000  0.300000   
9   4029749871687083265   3662.136548  0.031705  0.719655  0.000000  0.428738   
21  3891020560590657281   2856.300564  0.017434  1.129692  0.000000  0.647597   
38  4036548677626662656   3296.876680  0.005617  0.736671  0.000000  0.462585   

    Win_Positions  Total_Positions      Score  
35           2780             3554  16.035181  
60              6               20  11.595240  
9             367              856   4.842235  
21            283              437   4.651024  
38             68              147   4.501749  

Top 20 Accounts Based on Composite Score:
                 Port_ID           PnL       ROI    Sharpe       MDD  \
35   3986814617275053313  16337.461881  0.008716  1.288733  2.381947   
60   3936